In [ ]:
import sys

from ipywidgets import Button, Dropdown, Output, HBox, VBox

from ipyleaflet import Map, MapStyle, Marker

from bqplot import Axis, LinearScale, Lines, Figure, Toolbar

from precipitation.predicted import rainfall_pred1 as prcp

In [ ]:
duration_dropdown = Dropdown(description='Duration', value='30-min', options=[
    '5-min', '10-min', '15-min', '30-min', '60-min',
    '2-hr', '3-hr', '6-hr', '12-hr', '24-hr',
    '2-day', '3-day', '4-day', '7-day', '10-day',
    '20-day', '30-day', '45-day', '60-day'
])

annual_return_intervals_dropdown = Dropdown(description='Return', value='all', options=[
    '1', '2', '5', '10', '25', '50',
    '100', '200', '500', '1000', 'all'
])

message = Output()

In [ ]:
# Create map and marker for the precipitaion location
m = Map(center=(37.09, -95.71), zoom=4, scroll_wheel_zoom=True)

marker = Marker(location=(37.09, -95.71), draggable=True)

m.add_layer(marker)

m

In [ ]:
# On click on the map: Update marker location
def on_interaction(event, type, coordinates):
    global marker

    if type == 'click':
        marker.location = coordinates

m.on_interaction(on_interaction)

In [ ]:
# Create 2-D plot for the line chart
sc_x = LinearScale()
sc_y = LinearScale()

ax_x = Axis(scale=sc_x, label='Minutes', tick_format='.1f')
ax_y = Axis(scale=sc_y, orientation='vertical', label='Inches', tick_format='i')

lines = Lines(x=[], y=[], scales={'x': sc_x, 'y': sc_y})

figure = Figure(marks=[lines], axes=[ax_x, ax_y], title='Precipitation')

toolbar = Toolbar(figure=figure)

In [ ]:
# Update the plot on change events
def update_plot(*args, **kwargs):
    global duration_dropdown
    global annual_return_intervals_dropdown
    global marker

    message.clear_output()

    lat, long = marker.location
    intervals = annual_return_intervals_dropdown.value
    duration = duration_dropdown.value
    
    with message:
        try:
            prcp.run_precip(lat, long, intervals, duration)

            print('Lat/Long: ({}, {})'.format(lat, long))

            mins = [i * 5 for i in prcp.hrs]

            lines.x = mins
            lines.y = prcp.list_return
        except:
            print('Marker should be in the USA', file=sys.stderr)

            lines.x = []
            lines.y = []

# Update plot when the marker moved, or when an option changed
marker.observe(update_plot, 'location')
duration_dropdown.observe(update_plot, 'value')
annual_return_intervals_dropdown.observe(update_plot, 'value')

In [ ]:
HBox((duration_dropdown, annual_return_intervals_dropdown, message))

In [ ]:
VBox((figure, toolbar))

In [ ]:
update_plot()